<a href="https://colab.research.google.com/github/teddylew12/Waymo_Open_DataSet/blob/master/Generate_Weather_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/data')


In [0]:
!pip3 install waymo-open-dataset
!pip3 install tensorflow==2.0.0
import os
import tensorflow as tf
import math
import numpy as np
import itertools
import random
from PIL import Image
from tqdm import tqdm


from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset


In [0]:
# Change the CD To your specific path
DIRNAME ="/content/drive/Shared drives/Waymo Project/Data/CleanTest/weather_images"
os.chdir(DIRNAME)
! mkdir weather_images

In [0]:

def getWeather(FILENAME):
  dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
  data = dataset.take(1)
  frame = open_dataset.Frame()
  frame.ParseFromString(bytearray(data.numpy()))
  return  frame.context.stats.weather
def generate_file_list(dirname):
  sun_files =[]
  rain_files =[]
  for fname in os.listdir(dirname):
    if not fname.endswith(".tfrecord"):
      continue
    fullname = os.path.join(dirname,fname)
    if getWeather(fullname) == "sunny":
      sun_files.append(fname)
    else:
      rain_files.append(fname)
  #There will be an imbalance (lots more sunny files), so lets make them equal
  num_files = len(rain_files)
  print(num_files)
  sun_files = random.shuffle(sun_files)
  sun_files = sun_files[:num_files]
  assert(len(sun_files)==len(rain_files))
  return sun_files, rain_files

sun_files,rain_files = generate_file_list(DIRNAME)


In [0]:
def save_image(image,savename):
  decoded=tf.image.decode_jpeg(frame.images[i].image)
  tonumpy=decoded.numpy()
  im=Image.fromarray(tonumpy)
  name="sunny"+str(count)+ ".png"
  im.save(os.path.join('weather_images/',name))

def create_new_images(sun_arr, rain_arr):

  count = 0
  for i in tqdm(range(len(sun_arr))):
    dataset=tf.data.TFRecordDataset(sun_arr[i],compression_type='')
    for data in dataset:
      frame=open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))
      for i in range(len(frame.images)):
        if i not in [2,4]:
          save_name = "sunny"+str(count)+ ".png"
          save_image(frame.images[i].image,save_name)
          count += 1
  count = 0
  for i in tqdm(range(len(rain_arr))):
    dataset=tf.data.TFRecordDataset(rain_arr[i],compression_type='')
    for data in dataset:
      frame=open_dataset.Frame()
      frame.ParseFromString(bytearray(data.numpy()))
      for i in range(len(frame.images)):
        if i not in [2,4]:
          save_name = "rainy"+str(count)+".png"
          save_image(frame.images[i].image, save_name)
          count += 1

create_new_images(sun_files, rain_files)
